In [ ]:
from urllib import request
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

In [ ]:
def format_job(job):
    job_arr = job.split(' ')
    formatted_job = '-'.join(job_arr)
    return formatted_job

def get_all_soup(job_keyword):
    soup_list = []
    
    # scrapping the very first page
    job = format_job(job_keyword)
    url = f'https://www.jobstreet.com.sg/en/job-search/{job}-jobs/1/?job-type=internship'
    
    r = requests.get(url)
    print('1', r.status_code)
    soup = BeautifulSoup(r.content, 'html.parser')
    soup_list.append(soup)
    
    # extract the number of jobs related to this job keyword, to find out the number of pages of jobs this keyword has
    num_of_jobs_info = soup.find('span', {'class': 'sx2jih0 zcydq84u _18qlyvc0 _18qlyvc1x _18qlyvc1 _1d0g9qk4 _18qlyvc8'}).text
    stop_words = ['of', 'jobs']
    first = num_of_jobs_info.find(stop_words[0])
    second = num_of_jobs_info.find(stop_words[1])
    num_of_jobs = num_of_jobs_info[first+3: second-1]
    num_of_pages = int(num_of_jobs)//30 + 1
    print('num of jobs:', num_of_jobs)
    print('num of pages:', num_of_pages)
    
    # get all the soup for all the pages related to this job keyword
    for page in range(2, num_of_pages+1):
        url1 = f'https://www.jobstreet.com.sg/en/job-search/{job}-jobs/{page}/?job-type=internship'
        print(page, url1)
        r1 = requests.get(url1)
        print(page, r1.status_code)
        soup1 = BeautifulSoup(r1.content, 'html.parser')
        soup_list.append(soup1)
        time.sleep(3)
        
    return soup_list    

In [ ]:
base_url = 'https://www.jobstreet.com.sg'
def get_url(soup_list):
    url_list = []
    job_title_list = []
    for soup in soup_list:
        divs = soup.find_all('div', class_ = 'sx2jih0 zcydq876 zcydq866 zcydq896 zcydq886 zcydq8n zcydq856 zcydq8f6 zcydq8eu')
        for item in divs:
            job_title = item.find('span', class_ = 'sx2jih0').text
            job_title_list.append(job_title)
            job_url = item.find('a', href = True)['href']
            job_full_url = base_url + job_url
            url_list.append(job_full_url)
    return (url_list, job_title_list)
    # return url_list

In [ ]:
# additional codes to extract the extra job information
stop_points = ["Career Level", "Qualification", "Job Type", "Job Specializations"]
stop_point_string_lengths = [len(stop_point) for stop_point in stop_points]

# extracting the relevant details given a url
def get_job_details(url):
    r = requests.get(url)
    print(r.status_code)
    s = BeautifulSoup(r.content, 'html.parser')
    
    # get the job title
    job_title = s.h1.text
    #title = item.find('span', class_ = 'sx2jih0').text
    
    # get the company name
    company = s.find('div', {'class': 'sx2jih0 zcydq86i'}).find('span').text
    
    # get the work location
    location = s.find('div', {'class': 'sx2jih0 zcydq86a'}).text
    
    # get job information
    job_information = s.find('div', {'class': 'sx2jih0 zcydq86q zcydq86v zcydq86w'}).text
    job_information_index_to_slice = job_information.find('Additional Information')
    job_information = job_information[15:job_information_index_to_slice] #start from 15 to slice out 'Job Information' 
    ## HERE MIGHT HAVE ERROR, IF YES, GO BACK TO job_information[:job_information_index_to_slice]
    
    # get sector/indsutry
    company_extra_info = s.find_all('div', {'class': 'sx2jih0 _17fduda0 _17fduda7'})[1].text
    first = company_extra_info.find('Industry')
    second = company_extra_info.find('Benefit')
    sector = company_extra_info[first+8:second]
    
    # get additional information - career level, qualification, years of experience, job type and specialisation
    jai = s.find('div', {'class': 'sx2jih0 _17fduda0 _17fduda7'}).text
    indices = [jai.find(stop_points[i]) for i in range(len(stop_points))]
    jai_results = {}
    for i, stop_point in enumerate(stop_points):
        if i == len(stop_points)-1:
            extract = jai[indices[i] + stop_point_string_lengths[i] :]
        else:
            extract = jai[indices[i] + stop_point_string_lengths[i]: indices[i+1]]
        jai_results[stop_point] = extract
    
    # get salary
    test_salary = s.find_all('div', {'class': 'sx2jih0 zcydq86a'})[1].text
    if 'SGD' in test_salary:
        pay_range = test_salary
    else:
        pay_range = ''
    '''    
    job = {
        'job_title': job_title,
        'company': company,
        'location': location,
        'description': job_information,
        'level': jai_results['Career Level'],
        'qualification': jai_results['Qualification'],
        'job_type': jai_results['Job Type'],
        'job_specialisation': jai_results['Job Specializations'],
        'pay_range': pay_range
          }
    '''
    job = (url, job_title, company, location, sector, jai_results['Job Type'], jai_results['Career Level'], 
           jai_results['Job Specializations'], jai_results['Qualification'], pay_range, job_information)
    
    return job

In [ ]:
keyword_list = [
    'data',
    'tech' ,
    'software',
    'product manager',
    'analyst',
    'analytics',
    'data scientist',
    'data engineer',
    'artificial intelligence',
    'machine learning' ,
    'quantitative',
    'business intelligence',
    'computer vision',
    'natural language processing',
    'data pipelines'
]

In [ ]:
results = set()

def extract_jobstreet(keywords):
    jobs_scraped = 0
    for keyword in keywords:
        print(f'Current Keyword is: {keyword}')
        soups = get_all_soup(keyword)
        url_list = get_url(soups)[0]
        curr_job_count = 0
        for i, url in enumerate(url_list):
            print(i, url)
            results.add(get_job_details(url))
            time.sleep(5)
            jobs_scraped += 1
            curr_job_count += 1
            print('Number of jobs scraped :', jobs_scraped)
            print(keyword, ":", curr_job_count)
    return

In [ ]:
extract_jobstreet(['business intelligence'])

In [ ]:
col_list = ['url', 'job_title', 'company', 'location', 'sector', 'job_type', 'level', 'job_specialisation', 'qualification', 'pay_range', 'description']
df = pd.DataFrame(results, columns = col_list)
# df.head()
df.to_csv('jobstreet.csv')